# Fine-tuning a T5-small model for summarization

Here, we target the fine-tuning, the quantization and the evaluation parts of the practical use case.

I chose to work on summarization as it is a task we all need from time to time, or even daily. Moreover, several datasets are available on HuggingFace. I will be using the [XSum](https://arxiv.org/pdf/1808.08745) dataset available on [HuggingFace](https://huggingface.co/datasets/EdinburghNLP/xsum), which contains BBC articles accompanied with single-sentence summaries.

Now, let's talk about the chosen model. The T5 model was presented in [Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer](https://arxiv.org/pdf/1910.10683.pdf) by Colin Raffelet al. and is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks and for which each task is converted into a text-to-text format. T5 works well on a variety of tasks out-of-the-box by prepending a different prefix to the input corresponding to each task, e.g., for translation or summarization. In our case, the prefix for summarization task will be `summarize: `. It uses relative scalar embeddings and comes in different sizes: small, base, large, 3b, 11b.

I chose the T5-small model for three reasons:

*   T5 models are very well suited for summarization tasks ;
*   I am familiar with this family of model ;
*   From a practical point of view, I fine-tuned the model using a T4 GPU from Google Colab and the T5-small is the smallest model possible.

To fine-tune the model, I use LoRA, or Low Rank Adaptation, from the PEFT (Parameter Efficient Fine-Tuning) family. It is a technique that accelerates the fine-tuning of large models while consuming less memory, which is something valuable in my Google Colab environment! The idea is to freeze the original pre-trained weights and introduce an additional matrix computed using matrix decomposition, which makes it small. This new matrix is trained on new data while the original weights matrix are frozen, thus reducing the number of trainable parameters. Finally, both the original and the adapted weights are combined.

To evaluate the fine-tuned model and compare it with the original one, I chose the ROUGE metric. [ROUGE](https://huggingface.co/spaces/evaluate-metric/rouge), or Recall-Oriented Understudy for Gisting Evaluation, is used for evaluating automatic summarization and machine translation software in Natural Language Processing (NLP). In our case, it will compare an automatically produced summary with a human-produced (set of) reference(s). ROUGE is case insensitive and consists of 4 different metrics (depending on the length of the subsequence you base your score).

## Requirements and setup

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CDI/LLM_valeo/training')

Mounted at /content/drive


In [2]:
!pip install datasets evaluate transformers rouge-score nltk peft

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=aae7fcfa65a62b410f94e65f6664dbf406d3ad882c2256195f777d0327ae8900
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyar

In [3]:
import numpy as np
import nltk
import transformers
from datasets import load_dataset
from evaluate import load
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model

assert transformers.__version__ >= "4.11.0"
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## T5-small summarization fine-tuning with LoRA

### Load the data

We will use the HuggingFace [Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation.

In [4]:
raw_dataset = load_dataset("xsum", trust_remote_code=True)
metric = load("rouge")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Let's see some excerpts of the dataset:

In [5]:
print("XSum dataset summary:")
print(raw_dataset)

print("\nXSum dataset sample:")
raw_dataset["train"][0]

XSum dataset summary:
DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

XSum dataset sample:


{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

### Pre-process the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a HuggingFace Transformers `Tokenizer` which will tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, I instantiate a tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- the tokenizer corresponds to the T5-small model architecture ;
- the vocabulary used when pretraining this specific checkpoint is downloaded.

In [6]:
model_checkpoint = "t5-small"
prefix = "summarize: "

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Then, the inputs are fed to the tokenizer with the argument `truncation=True`, that will ensure that an input is not longer that what the model selected can handle. Otherwise, the input will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [7]:
max_input_length = 1024
max_target_length = 128

def preprocessing(sample, prefix="summarize: "):
    inputs = [prefix + doc for doc in sample["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = raw_dataset.map(preprocessing, batched=True)

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

### LoRA fine-tuning

 Let's download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class.

In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

For the LoRA fine-tuning, we need to freeze the parameters of the model.

In [9]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

In [10]:
config = LoraConfig(
    r=4, # dimension of the low-rank matrices
    lora_alpha=32, # scaling factor for the low-rank matrices
    lora_dropout=0.05, # dropout probability of the LoRA layers
    target_modules=["k","q","v","o"],
    bias="none",
    task_type="Seq2Seq" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


We need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels

In [11]:
batch_size = 32

training_args = Seq2SeqTrainingArguments(
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=2, # equivalent to batch_size = 64
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=100,
    num_train_epochs=1,
    # fp16=True,
    evaluation_strategy="steps",
    eval_steps=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    predict_with_generate=True,
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=3,
    output_dir="outputs"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We need to define a function to compute the ROUGE metric, which will just use the metric we loaded earlier. We also have to do a bit of pre-processing to decode the predictions into texts.

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Now, we can start the fine-tuning.

In [13]:
trainer = Seq2SeqTrainer(
    model=model,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
319,No log,2.986775,18.073300,2.750000,14.175700,14.331200,18.772500
638,3.457900,2.794375,20.729500,3.763200,16.045400,16.048700,18.535600


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args

In [ ]:
trainer.train(resume_from_checkpoint=True)

/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3087: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
957,3.457900,2.743776,22.424500,4.342000,17.295000,17.289900,18.561300


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Let's save the LoRA checkpoint.

In [ ]:
model.save_pretrained("outputs/lora-xsum-t5-small")


### Evaluation

Now, we use the test split to compare the fine-tuned model with the base one.

In [ ]:
# prompt: Evaluate the model using the test set of the tokenized_dataset and HuggingFace, use the ROUGE metric

# Evaluate the model using the test set of the tokenized_dataset and HuggingFace, use the ROUGE metric
results = trainer.evaluate(tokenized_dataset['test'])
print(results)
